영-일 dictionary(https://www.romajidesu.com/dictionary/meaning-of-rice.html#sentences)에서  
최대한 많은 parallel 데이터를 수집하고, 정제한 후 jsonl file에 저장하는 코드를 작성

In [1]:
import requests
from bs4 import BeautifulSoup
import json 

#4가지 단어들을 검색 : rice, clothes, house, eat
rice_url = "https://www.romajidesu.com/dictionary/meaning-of-rice.html#sentences"
clothes_url ="https://www.romajidesu.com/dictionary/meaning-of-clothes.html"
house_url ="https://www.romajidesu.com/dictionary/meaning-of-house.html"
eat_url = "https://www.romajidesu.com/dictionary/meaning-of-easy.html"

url_collection=[rice_url, clothes_url, house_url, eat_url]

# DEFINITION of 검색어 (matched results)

In [2]:
def get_definition_data(soup, row_data):
    search_result = soup.find('div', class_ = 'search_result')
    search_items = search_result.find_all('div', class_ ='search_items')
    
    for item in search_items[:]:
        
        #품사(pos)
        pos_list = item.find('span', class_='pos')
        POS = pos_list.find('a').text
        
        # 뜻 
        meaning_list = item.find_all('span', class_='gloss')
        MEANING_LIST = []
        for meaning in meaning_list:
            MEANING_LIST.append(meaning.text)
        
        #연관단어
        related_word = item.find('span', class_='related')
        if related_word: #존재한다면
            related_word = related_word.find('a').text
        
        #kanji한자 관련
        KANJ_LIST=[]
        
        kanj_items = item.find('span', class_='word_kanj')        
        if kanj_items:#존재한다면
            kanj_list = kanj_items.find_all('a')
            for kanj in kanj_list:
                KANJ_LIST.append(kanj.text)
        else:
            KANJ_LIST= None
        
        # 가타카나 ,발음 관련
        KANA_WORD_LIST=[]
        KANA_PRONUNCIATE_LIST=[]
        
        kana_item = item.find('span', class_='word_kana')
        kana_list = kana_item.find_all('ruby')
        
        for kana in kana_list:
            KANA_WORD_LIST.append(kana.find('rb').text) #가타카나 단어
            KANA_PRONUNCIATE_LIST.append(kana.find('rt').text) #가타카나 발음
        
        matched_definition ={
            'POS' : POS,
            'MEANING' : MEANING_LIST,
            'related_word' : related_word,
            'KANJ' : KANJ_LIST,
            'KANA_WORD': KANA_WORD_LIST,
            'KANA_PRONUNCIATION': KANA_PRONUNCIATE_LIST
        }

        print(matched_definition, '\n')
        row_data.append(matched_definition)

# WORDS RELATED TO 검색어 (유사의미)

In [3]:
def get_related_words_data(soup, row_data):
    related_words = soup.find(id = 'related_words')
    related_items = related_words.find_all('div', class_ = 'search_items')
    
    for item in related_items[:]:
        #품사(pos)
        pos_list = item.find('span', class_='pos')
        POS= pos_list.find('a').text #괄호 제거 용도

        # 뜻 
        meaning_list = item.find_all('span', class_='gloss')
        MEANING_LIST = []
        for meaning in meaning_list:
            MEANING_LIST.append(meaning.text)
    
        #연관단어
        related_word = item.find('span', class_='related')
        if related_word: #존재한다면
            related_word = related_word.find('a').text
    
        #kanji한자 관련
        KANJ_LIST=[]
        kanj_items = item.find('span', class_='word_kanj')
    
        if kanj_items:#존재한다면
            kanj_list = kanj_items.find_all('a')
            for kanj in kanj_list:
                KANJ_LIST.append(kanj.text)
        else:
            KANJ_LIST= None 

        # 가타카나 , 가타카나 발음 관련            
        KANA_WORD_LIST=[]
        KANA_PRONUNCIATE_LIST=[]
        
        kana_item = item.find('span', class_='word_kana')
        kana_list = kana_item.find_all('ruby')

        for kana in kana_list:
            KANA_WORD_LIST.append(kana.find('rb').text) #가타카나 단어
            KANA_PRONUNCIATE_LIST.append(kana.find('rt').text) #가타카나 발음
        
        related_definition={
            'POS' : POS,
            'MEANING': MEANING_LIST,
            'related_word': related_word,
            'KANJ' : KANJ_LIST,
            'KANA_WORD': KANA_WORD_LIST,
            'KANA_PRONUNCIATION' : KANA_PRONUNCIATE_LIST
        }
        print(related_definition,'\n')
        row_data.append(related_definition)

# Sentences containing 검색어


In [4]:
def get_sentences(soup, row_data):
    related_examples = soup.find('div', id="related_examples")
    related_examples = related_examples.find_all('div', class_='examples')
    
    for example in related_examples[:]:
        
        #pronunciation example
        jpn_easy_ex = example.find_all('rt') 
        PRONUNCIATION_EX =''
        for easy in jpn_easy_ex:
            PRONUNCIATION_EX+= (easy.text)
    
        #original example 
        jpn_ex = example.find('div', 'eg_jap')
        ORIGINIAL_EX = jpn_ex.text.strip()

        
        #english examples
        eng_ex = example.find('div', class_='eg_eng')
        ENG_EX = eng_ex.text.strip()
        
        sentence_example={
            'PRONUNCIATION' : PRONUNCIATION_EX,
            'EXAMPLE': ORIGINIAL_EX,
            'ENGLISH' : ENG_EX
        }
        print(sentence_example)
        row_data.append(sentence_example)

In [5]:
##main

row_data =[] #line들을 여기에 저장 

for url in url_collection[:]:
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    eng_entry = soup.find('em').get_text()
    print('searched eng entry is:', eng_entry) #영어검색어
    
    get_definition_data(soup, row_data)
#     get_related_words_data(soup, row_data) #오역이 많아 제외함
    get_sentences(soup, row_data)

searched eng entry is: rice
{'POS': 'n', 'MEANING': ['rice (esp. when served on a plate)'], 'related_word': 'ライスカレー', 'KANJ': None, 'KANA_WORD': ['ライス'], 'KANA_PRONUNCIATION': ['raisu']} 

{'POS': 'n', 'MEANING': ['(white) rice'], 'related_word': None, 'KANJ': ['白米'], 'KANA_WORD': ['はくまい'], 'KANA_PRONUNCIATION': ['hakumai']} 

{'POS': 'n', 'MEANING': ['(husked grains of) rice', '88 years old'], 'related_word': None, 'KANJ': ['米'], 'KANA_WORD': ['こめ', 'よね', 'めめ', 'コメ'], 'KANA_PRONUNCIATION': ['kome', 'yone', 'meme', 'kome']} 

{'POS': 'n', 'MEANING': ['rice'], 'related_word': None, 'KANJ': ['米穀'], 'KANA_WORD': ['べいこく'], 'KANA_PRONUNCIATION': ['beikoku']} 

{'POS': 'n', 'MEANING': ['rice'], 'related_word': None, 'KANJ': ['飯米'], 'KANA_WORD': ['はんまい'], 'KANA_PRONUNCIATION': ['hanmai']} 

{'POS': 'n', 'MEANING': ['(husked grains of) rice'], 'related_word': '米', 'KANJ': ['お米'], 'KANA_WORD': ['おこめ'], 'KANA_PRONUNCIATION': ['okome']} 

{'POS': 'n', 'MEANING': ['rice (as provisions)'], 'related

searched eng entry is: easy
{'POS': 'adj-f', 'MEANING': ['simple; light; easy; openhearted'], 'related_word': None, 'KANJ': None, 'KANA_WORD': ['あっさりした'], 'KANA_PRONUNCIATION': ['assarishita']} 

{'POS': 'adj-i', 'MEANING': ['easy; simple', 'soft; lax; slack'], 'related_word': None, 'KANJ': None, 'KANA_WORD': ['ちょろい'], 'KANA_PRONUNCIATION': ['choroi']} 

{'POS': 'adj-na', 'MEANING': ['easy'], 'related_word': None, 'KANJ': None, 'KANA_WORD': ['イージー'], 'KANA_PRONUNCIATION': ['iijii']} 

{'POS': 'adj-na', 'MEANING': ['easy; simple', 'easy-going'], 'related_word': None, 'KANJ': ['安易'], 'KANA_WORD': ['あんい'], 'KANA_PRONUNCIATION': ['ani']} 

{'POS': 'adj-na', 'MEANING': ['inexpensive; cheap; easy; simple'], 'related_word': None, 'KANJ': ['安直'], 'KANA_WORD': ['あんちょく'], 'KANA_PRONUNCIATION': ['anchoku']} 

{'POS': 'adj-i', 'MEANING': ['easy'], 'related_word': None, 'KANJ': ['易い'], 'KANA_WORD': ['やすい'], 'KANA_PRONUNCIATION': ['yasui']} 

{'POS': 'adj-i', 'MEANING': ['easy; plain; simple'], 'rel

In [6]:
len(row_data)

109

In [7]:
row_data

[{'POS': 'n',
  'MEANING': ['rice (esp. when served on a plate)'],
  'related_word': 'ライスカレー',
  'KANJ': None,
  'KANA_WORD': ['ライス'],
  'KANA_PRONUNCIATION': ['raisu']},
 {'POS': 'n',
  'MEANING': ['(white) rice'],
  'related_word': None,
  'KANJ': ['白米'],
  'KANA_WORD': ['はくまい'],
  'KANA_PRONUNCIATION': ['hakumai']},
 {'POS': 'n',
  'MEANING': ['(husked grains of) rice', '88 years old'],
  'related_word': None,
  'KANJ': ['米'],
  'KANA_WORD': ['こめ', 'よね', 'めめ', 'コメ'],
  'KANA_PRONUNCIATION': ['kome', 'yone', 'meme', 'kome']},
 {'POS': 'n',
  'MEANING': ['rice'],
  'related_word': None,
  'KANJ': ['米穀'],
  'KANA_WORD': ['べいこく'],
  'KANA_PRONUNCIATION': ['beikoku']},
 {'POS': 'n',
  'MEANING': ['rice'],
  'related_word': None,
  'KANJ': ['飯米'],
  'KANA_WORD': ['はんまい'],
  'KANA_PRONUNCIATION': ['hanmai']},
 {'POS': 'n',
  'MEANING': ['(husked grains of) rice'],
  'related_word': '米',
  'KANJ': ['お米'],
  'KANA_WORD': ['おこめ'],
  'KANA_PRONUNCIATION': ['okome']},
 {'POS': 'n',
  'MEANING':

In [8]:
with open('parallel_data.jsonl', encoding='utf-8', mode='w') as file:
    for line in row_data:
        file.write(json.dumps(line, ensure_ascii=False)+ '\n')

In [9]:
'''
#파일 읽기 (검토용)
check=[]
with open('./최종과제_제출물/parallel_data.jsonl','r',encoding='utf-8') as file:
    for line in file:
        check.append(json.loads(line))
len(check)
'''

"\n#파일 읽기 (검토용)\ncheck=[]\nwith open('./최종과제_제출물/parallel_data.jsonl','r',encoding='utf-8') as file:\n    for line in file:\n        check.append(json.loads(line))\nlen(check)\n"